In [266]:
#Load some useful modules
import os
import numpy as np
import gzip, struct
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import pickle, gzip
import scipy.io as sio


# Load the raw files into a numpy array

In [267]:
personId="1"
eeg_s1 = sio.loadmat('./Eyes-closed/Session 2/S'+personId+'_SR200_CE_EEG.ch.mat')
eeg_s1['EEG']['samp'][0][0].shape

(12000, 56)

In [268]:
mpl.get_backend()

'agg'

In [269]:
plt.clf()
plt.cla()
fig_size = plt.rcParams["figure.figsize"]
print "Current size:", fig_size
fig_size[0] = 10
fig_size[1] = 2
plt.rcParams["figure.figsize"] = fig_size
fig = plt.figure()
subEEG = eeg_s1['EEG']['samp'][0][0][:601]
plt.yticks([])
plt.imshow(subEEG.T)
plt.tight_layout()
plt.savefig("graph/eeg"+personId+".pdf",dpi=600)

Current size: [15.0, 25.0]


# Dataset Description

In [270]:
EEG=eeg_s1['EEG']['samp'][0][0]
EEG.shape

(12000, 56)

#Statistics

In [271]:
np.min(np.amin(EEG, axis=0).reshape(8,7)) 

-129.20000000000002

In [272]:
np.max(np.amax(EEG, axis=0).reshape(8,7))

188.80000000000001

In [273]:
np.round(np.mean(EEG, axis=0).reshape(8,7),2)

array([[ 38.4 ,  33.64,  35.66,   3.26,  16.14,  17.06,  10.57],
       [ 26.19,  25.81,  15.87,  -6.38,  20.17,  14.17,  37.89],
       [ 19.54,  16.21,  21.47,  12.59,  28.31,  12.01,  10.67],
       [  9.15,   4.31,  12.58,  -4.2 ,  15.1 ,  17.21,  -8.13],
       [ 18.68,  12.29,  -4.8 ,  -0.09,  14.78,   9.1 ,  13.77],
       [  4.69,   8.81,   0.35,  16.04,  29.93,  16.08,  14.51],
       [  8.82,  14.28,  13.87,   3.67,  12.64,   6.38,   1.72],
       [  4.07,  10.47,  10.3 ,  10.03,   4.88,   8.28,   4.  ]])

In [274]:
np.round(np.std(EEG, axis=0).reshape(8,7),2)

array([[ 40.61,  50.21,  45.71,  32.84,  36.74,  37.35,  27.48],
       [ 29.16,  28.99,  28.34,  25.47,  28.14,  30.07,  33.48],
       [ 47.24,  22.31,  26.02,  22.71,  33.57,  30.83,  25.92],
       [ 27.64,  17.24,  18.62,  25.67,  23.56,  22.33,  22.17],
       [ 21.38,  20.26,  18.17,  17.01,  19.87,  23.36,  23.34],
       [ 23.55,  22.64,  22.01,  25.13,  20.4 ,  21.24,  23.7 ],
       [ 27.34,  27.54,  24.62,  25.54,  24.39,  21.15,  16.38],
       [ 26.94,  26.99,  28.38,  25.53,  25.22,  24.97,  25.31]])

In [275]:
plt.clf()
plt.cla()
fig_size[0] = 15
fig_size[1] = 25
plt.rcParams["figure.figsize"] = fig_size
fig = plt.figure()
for i in range(EEG.shape[1]):
    ax = fig.add_subplot(8, 7, i+1)
    plt.hist(EEG[:,i],50, range=[-355, 104],normed=1)
    ax.get_xaxis().set_ticks([-355, 0., 183])
    ax.get_yaxis().set_ticks([])
plt.show()
plt.tight_layout()
plt.savefig("graph/eeg_hist"+personId+".pdf",dpi=600)

In [276]:
EEG = EEG.astype(float)

In [277]:
EEG = (EEG - np.amin(EEG))/(np.amax(EEG) - np.amin(EEG))

In [278]:
#train_perm = np.random.permutation(EEG.shape[0])
#print(np.max(train_perm))
#EEG = EEG[train_perm,:]

In [279]:
EEG

array([[ 0.62704403,  0.6009434 ,  0.5836478 , ...,  0.5       ,
         0.49559748,  0.50188679],
       [ 0.63081761,  0.61037736,  0.59056604, ...,  0.50754717,
         0.49433962,  0.52232704],
       [ 0.57012579,  0.55503145,  0.5418239 , ...,  0.46037736,
         0.43584906,  0.47704403],
       ..., 
       [ 0.18459119,  0.16257862,  0.12075472, ...,  0.33805031,
         0.40314465,  0.37421384],
       [ 0.18930818,  0.16540881,  0.12515723, ...,  0.33176101,
         0.3990566 ,  0.37075472],
       [ 0.22327044,  0.19654088,  0.16037736, ...,  0.36163522,
         0.42924528,  0.39968553]])

In [280]:
# Write the data
newFile = open ("Eyes-closed/Session 2/EEG"+personId+".dat", "wb")
newFile.write(struct.pack('i', EEG.shape[0]))
newFile.write(struct.pack('i', EEG.shape[1]))
EEG.astype('float').tofile(newFile)
newFile.close()